In [1]:
import pickle
import pprint
import glob
import pandas as pd
import seaborn as sns
pp = pprint.PrettyPrinter(depth=6)
pd.set_option('max_colwidth',400)

In [2]:
csv_folder = "../csv"

In [3]:
fields = {
    'id': None,
    "build": ["id"],
 'started_at': None,
"created_at":None,
'finished_at': None,
 'number': None,
 'state': None,
 'updated_at': None,
"commit": ["sha", "id", "message", "ref", "compare_url"],}

### Import dataframes

In [4]:
allBuilds = pd.read_csv("../csv/allBuilds.csv", index_col=0)

In [5]:
pr_commits = pd.read_csv("../csv/pr_commits.csv", index_col=0)
pr_commits

,pr_number,commitsha
0,2016,66dcca6c302a3af95f5b7d3ca269588680bf8a82
1,2029,da3dff653d07655d6ebecb1d869f6bd93a8763c9
2,2029,c273e7ce2ac80d9f230f85b89f49e603b8d726ca
3,2029,0c9fca66dea0b8ab3825b0fe484188f07c5802e4
4,2029,5dfcbf4bd034c59b24b218dbe440c93346b99949
...,...,...
12363,616,52c69fcd007fcad58f23bc80ffcab9b06326fb0a
12364,602,8d809c371e9b00c1886b1f7e83d8699e546bdb0e
12365,621,347978968e87735dacb333f72e32f20b8a7a3163
12366,621,f1b4428c014da7768d14bbd34cc322594b283bb4


### Import jobs

In [6]:
job_files = glob.glob("../jobs/*.pkl")
jobs = []
for job_file in job_files:
    with open(job_file, "rb") as f:
        jobs = jobs + pickle.load(f)

In [7]:
len(jobs)

136983

In [8]:
pp.pprint(jobs[0])

{'@href': '/job/241946101',
 '@permissions': {'cancel': False,
                  'debug': False,
                  'delete_log': False,
                  'read': True,
                  'restart': False},
 '@representation': 'standard',
 '@type': 'job',
 'allow_failure': False,
 'build': {'@href': '/build/241946099',
           '@representation': 'minimal',
           '@type': 'build',
           'duration': 40,
           'event_type': 'push',
           'finished_at': '2017-06-12T09:21:43Z',
           'id': 241946099,
           'number': '25614',
           'previous_state': 'canceled',
           'private': False,
           'pull_request_number': None,
           'pull_request_title': None,
           'started_at': '2017-06-12T09:21:03Z',
           'state': 'canceled'},
 'commit': {'@representation': 'minimal',
            '@type': 'commit',
            'committed_at': '2017-06-12T09:13:40Z',
            'compare_url': 'https://github.com/SonarSource/sonarqube/compare/faddae08a6

In [9]:
job_tuples = []
for job in jobs:
    job_values = []
    for field, subfields in fields.items():
        if not subfields:
            job_values.append(job[field])
            continue
        for subfield in subfields:
            job_values.append(job[field][subfield])
    job_tuples.append(tuple(job_values))

In [10]:
len(job_tuples)

136983

In [11]:
columns = []
for field, subfields in fields.items():
        if not subfields:
            columns.append(field)
            continue
        for subfield in subfields:
            columns.append(f"{field}{subfield}")

In [12]:
columns

['id',
 'buildid',
 'started_at',
 'created_at',
 'finished_at',
 'number',
 'state',
 'updated_at',
 'commitsha',
 'commitid',
 'commitmessage',
 'commitref',
 'commitcompare_url']

In [13]:
jobsDf = pd.DataFrame(job_tuples, columns = columns)
jobsDf.to_csv(f"{csv_folder}/allJobs.csv")
jobsDf

,id,buildid,started_at,created_at,finished_at,number,state,updated_at,commitsha,commitid,commitmessage,commitref,commitcompare_url
0,241946101,241946099,2017-06-12T09:21:03Z,2017-06-12T09:13:51.063Z,2017-06-12T09:21:43Z,25614.1,canceled,2018-06-02T14:51:19.734Z,2e4b539138fde6f664a865c81fb6ab0763f74bdb,70181895,SONAR-9304 Add test on organization deletion,refs/heads/feature/atlas/MMF-324/builtin-profiles,https://github.com/SonarSource/sonarqube/compare/faddae08a6af...2e4b539138fd
1,241946103,241946099,2017-06-12T09:22:43Z,2017-06-12T09:13:51.107Z,2017-06-12T09:21:43Z,25614.2,canceled,2018-06-02T14:51:19.764Z,2e4b539138fde6f664a865c81fb6ab0763f74bdb,70181895,SONAR-9304 Add test on organization deletion,refs/heads/feature/atlas/MMF-324/builtin-profiles,https://github.com/SonarSource/sonarqube/compare/faddae08a6af...2e4b539138fd
2,241952024,241952022,2017-06-12T09:36:43Z,2017-06-12T09:32:52.227Z,2017-06-12T09:37:23Z,25615.1,canceled,2018-06-02T14:51:25.661Z,464f5da96733c14818f4b4bfeb878951b859ee0c,70183628,SONAR-9304 Add comments to make tests clearer,refs/heads/feature/atlas/MMF-324/builtin-profiles,https://github.com/SonarSource/sonarqube/compare/2e4b539138fd...464f5da96733
3,241952025,241952022,None,2017-06-12T09:32:52.237Z,2017-06-12T09:37:24Z,25615.2,canceled,2018-06-02T14:51:25.669Z,464f5da96733c14818f4b4bfeb878951b859ee0c,70183628,SONAR-9304 Add comments to make tests clearer,refs/heads/feature/atlas/MMF-324/builtin-profiles,https://github.com/SonarSource/sonarqube/compare/2e4b539138fd...464f5da96733
4,241952045,241952044,2017-06-12T09:42:16Z,2017-06-12T09:32:59.994Z,2017-06-12T09:53:48Z,25616.1,passed,2018-06-02T14:51:26.125Z,7358e9a93e47ac9fcaebe960c00ab8bd28e9890e,70183633,SONAR-9304 Add comments to make tests clearer,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136978,315411337,315411335,2017-12-12T15:23:15Z,2017-12-12T15:22:41.091Z,2017-12-12T15:26:39Z,34811.2,passed,2018-06-03T16:57:36.971Z,abd0650045b3fcc95ac8459a6fc2450a1bd889c0,93642638,SONAR-10134 Fix ITs on quality gates by setting organization,refs/pull/2866/merge,https://github.com/SonarSource/sonarqube/pull/2866
136979,315421139,315421138,2017-12-12T15:43:05Z,2017-12-12T15:42:34.642Z,2017-12-12T15:44:59Z,34812.1,passed,2018-06-03T16:57:47.762Z,04d5e19040df59e2a4a1025514c9b8b7c42378f8,93645831,"SONAR-10187 Provide more options to populate empty ""My Projects"" page",refs/heads/feature/stas/MMF-1128,https://github.com/SonarSource/sonarqube/compare/d04d5cd97f40...04d5e19040df
136980,315421140,315421138,2017-12-12T15:43:26Z,2017-12-12T15:42:34.682Z,2017-12-12T15:44:25Z,34812.2,passed,2018-06-03T16:57:47.778Z,04d5e19040df59e2a4a1025514c9b8b7c42378f8,93645831,"SONAR-10187 Provide more options to populate empty ""My Projects"" page",refs/heads/feature/stas/MMF-1128,https://github.com/SonarSource/sonarqube/compare/d04d5cd97f40...04d5e19040df
136981,315421147,315421146,2017-12-12T15:43:25Z,2017-12-12T15:42:40.583Z,2017-12-12T16:01:05Z,34813.1,passed,2018-06-03T16:57:47.837Z,a5005e0ca38a67e7d70c255154990ed10be31020,93645838,"SONAR-10187 Provide more options to populate empty ""My Projects"" page",refs/pull/2895/merge,https://github.com/SonarSource/sonarqube/pull/2895


**Jobs with no commits**

In [14]:
jobsDf["commitsha"].isna().sum()

0

In [15]:
job_id_build_id = pd.read_csv("../csv/job_id_build_id.csv", index_col=0)

In [16]:
job_id_build_id

,build_id,build_number,job_id
0,99118995,6995,99118996
1,99118995,6995,99118997
2,99118995,6995,99118998
3,99118995,6995,99118999
4,99118995,6995,99119000
...,...,...,...
136978,148005355,12164,148005357
136979,148010751,12165,148010752
136980,148010751,12165,148010753
136981,148010754,12166,148010755


In [17]:
len(job_id_build_id.job_id.unique())

136983

In [18]:
x = job_id_build_id[~job_id_build_id.job_id.isin(jobsDf.id.unique())]
x

,build_id,build_number,job_id


We have all jobs!

## Pull request tests

In [19]:
pr_number = 2104
pull_req = f"refs/pull/{pr_number}/merge"

In [20]:
pr_sha = pr_commits[pr_commits.pr_number == pr_number].commitsha.unique()
pr_sha

array(['3618f6417ade2a02a139c99b2add0967ca647288',
       'b6dd52cf7cd8c47988cfdb9a9887ac4e66c61999',
       'd5bdf821a39621dbf4bd60dbb3412ea5c36b6ddc',
       'bab99c01f67f310636502eccbf9952e4e0ea42c1',
       '0fb3084e6c609fb9b94cb67ed7c7ac884502aea3',
       '71987dfa5a6c563b3d8acf86592517a61ade3bc7',
       'b376b9da30fea9abeedafe0c30b82f3b3a9f5207',
       '90c44d08cd1203c578b4c0f66126e3c22187c4ed',
       '702b6393c7b9e77886908bfd4565694b13ccdef8',
       'f7e2e9c191c8da127f2d0d8504ad0fa9f816191b',
       '14f35ba612ceca42d7ad6eb7092bfaf83dd7b9b3',
       'c23644d8c6f3e93f1ef6cda4ec973e2e9ded72c2',
       'ae85f66a8dbef6f7533cb6cb1eb9ab623ad8d447',
       'd4133c63eb8e7b6acc54195da3206f1b3a5f7689',
       '3e26c9b9b44ebabbc2c81e134c26799ce3f4bc3b',
       '380816fec9f40f8d2c25c3f08be2d783b7293226',
       '64d9dd615655021a0022e3a6f7cadbe7ab170231',
       '7122d530b75bb5f28f47c0a6ebf29e71012ce5cc',
       '4c9807a6d016bd7da4419f21c8a736f5f014493d',
       '93ece4c32cb488af46855b3

In [21]:
jobsDf[jobsDf.commitsha.isin(pr_sha)]

,id,buildid,started_at,created_at,finished_at,number,state,updated_at,commitsha,commitid,commitmessage,commitref,commitcompare_url
46956,242836986,242836985,2017-06-14T13:44:51Z,2017-06-14T13:43:49.953Z,2017-06-14T13:45:34Z,25802.1,canceled,2018-06-02T15:09:51.846Z,764185d9211229e5dcafe885cfdcbfbfadf54ce7,70450914,SONAR-9427 Improve performance on quality profile changelog,refs/heads/feature/atlas/MMF-324/builtin-profiles,https://github.com/SonarSource/sonarqube/compare/94592a6fb1f1...764185d92112
46957,242836987,242836985,2017-06-14T13:44:52Z,2017-06-14T13:43:50.009Z,2017-06-14T13:45:35Z,25802.2,canceled,2018-06-02T15:09:51.867Z,764185d9211229e5dcafe885cfdcbfbfadf54ce7,70450914,SONAR-9427 Improve performance on quality profile changelog,refs/heads/feature/atlas/MMF-324/builtin-profiles,https://github.com/SonarSource/sonarqube/compare/94592a6fb1f1...764185d92112
57967,243033842,243033841,None,2017-06-14T22:15:25.392Z,2017-06-14T22:16:17Z,25826.1,canceled,2018-06-02T15:15:03.029Z,b33d63ad5fb20e1b25ab377770fd91c6cb8f7cda,70509083,SONAR-9304 remove orphans from rules_profiles,refs/heads/feature/atlas/MMF-324/builtin-profiles,https://github.com/SonarSource/sonarqube/compare/2d5abe81787a...b33d63ad5fb2
57968,243033843,243033841,None,2017-06-14T22:15:25.474Z,2017-06-14T22:16:18Z,25826.2,canceled,2018-06-02T15:15:03.184Z,b33d63ad5fb20e1b25ab377770fd91c6cb8f7cda,70509083,SONAR-9304 remove orphans from rules_profiles,refs/heads/feature/atlas/MMF-324/builtin-profiles,https://github.com/SonarSource/sonarqube/compare/2d5abe81787a...b33d63ad5fb2
57971,243130913,243130912,2017-06-15T06:04:40Z,2017-06-15T06:03:19.992Z,2017-06-15T06:22:04Z,25828.1,passed,2018-06-02T15:17:07.922Z,b33d63ad5fb20e1b25ab377770fd91c6cb8f7cda,70538681,SONAR-9304 remove orphans from rules_profiles,refs/heads/master,https://github.com/SonarSource/sonarqube/compare/bcdb4a2044d6...b33d63ad5fb2
57972,243130914,243130912,2017-06-15T06:04:40Z,2017-06-15T06:03:20.045Z,2017-06-15T06:08:54Z,25828.2,passed,2018-06-02T15:17:08.007Z,b33d63ad5fb20e1b25ab377770fd91c6cb8f7cda,70538681,SONAR-9304 remove orphans from rules_profiles,refs/heads/master,https://github.com/SonarSource/sonarqube/compare/bcdb4a2044d6...b33d63ad5fb2


In [22]:
jobsDf[jobsDf.commitref == pull_req]

,id,buildid,started_at,created_at,finished_at,number,state,updated_at,commitsha,commitid,commitmessage,commitref,commitcompare_url
4,241952045,241952044,2017-06-12T09:42:16Z,2017-06-12T09:32:59.994Z,2017-06-12T09:53:48Z,25616.1,passed,2018-06-02T14:51:26.125Z,7358e9a93e47ac9fcaebe960c00ab8bd28e9890e,70183633,SONAR-9304 Add comments to make tests clearer,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
5,241952046,241952044,2017-06-12T09:42:15Z,2017-06-12T09:33:00.012Z,2017-06-12T09:46:56Z,25616.2,passed,2018-06-02T14:51:26.134Z,7358e9a93e47ac9fcaebe960c00ab8bd28e9890e,70183633,SONAR-9304 Add comments to make tests clearer,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
34,242001807,242001806,2017-06-12T12:32:57Z,2017-06-12T12:32:00.677Z,2017-06-12T12:44:03Z,25631.1,passed,2018-06-02T14:52:46.098Z,0b284e73ea86f4373e198549afbc27f12513d2e4,70199251,SONAR-9304 Add test on quality profiles for project analysis,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
35,242001808,242001806,2017-06-12T12:32:56Z,2017-06-12T12:32:00.683Z,2017-06-12T12:38:02Z,25631.2,passed,2018-06-02T14:52:46.106Z,0b284e73ea86f4373e198549afbc27f12513d2e4,70199251,SONAR-9304 Add test on quality profiles for project analysis,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
42,242002828,242002827,2017-06-12T12:35:54Z,2017-06-12T12:34:56.871Z,2017-06-12T12:47:22Z,25635.1,passed,2018-06-02T14:52:48.371Z,b492c6f375c836235b390a35b3c3caed77a89b47,70199512,(wip) replace RuleActivatorMediumTest by RuleActivatorTest,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135544,238061347,238061344,2017-05-31T19:40:36Z,2017-05-31T19:39:18.489Z,2017-05-31T19:44:46Z,25238.2,passed,2018-06-02T13:28:20.657Z,c990cd59bc753c19297e86b8404fc12798840aa5,68998200,SONAR-9344 Do not display activation actions for built-in quality profiles on rules page,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
135549,238073316,238073315,2017-05-31T20:14:35Z,2017-05-31T20:13:29.874Z,2017-05-31T20:27:10Z,25241.1,passed,2018-06-02T13:28:32.872Z,2c2c40df57223320530d4659a2cf4291cb39638f,69001951,SONAR-9304 drop columns organization_uuid and parent_kee from rules_profiles,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
135550,238073317,238073315,2017-05-31T20:14:31Z,2017-05-31T20:13:29.885Z,2017-05-31T20:19:01Z,25241.2,passed,2018-06-02T13:28:32.894Z,2c2c40df57223320530d4659a2cf4291cb39638f,69001951,SONAR-9304 drop columns organization_uuid and parent_kee from rules_profiles,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
135577,238275939,238275938,2017-06-01T09:48:06Z,2017-06-01T09:46:59.399Z,2017-06-01T10:00:36Z,25255.1,passed,2018-06-02T13:32:57.993Z,04eb3058e2a80375fe5536c34f23052482f1302d,69062205,DO NOT MERGE - Launch only PostgreSql QA on developer branch,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104


Many more jobs associated with that PR than the ones retrieved with the PR list of commits. Why?

**PR Jobs to investigate**

In [27]:
jobsDf[(jobsDf.commitref == pull_req) & ~(jobsDf.commitsha.isin(pr_sha))]

,id,buildid,started_at,created_at,finished_at,number,state,updated_at,commitsha,commitid,commitmessage,commitref,commitcompare_url
4,241952045,241952044,2017-06-12T09:42:16Z,2017-06-12T09:32:59.994Z,2017-06-12T09:53:48Z,25616.1,passed,2018-06-02T14:51:26.125Z,7358e9a93e47ac9fcaebe960c00ab8bd28e9890e,70183633,SONAR-9304 Add comments to make tests clearer,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
5,241952046,241952044,2017-06-12T09:42:15Z,2017-06-12T09:33:00.012Z,2017-06-12T09:46:56Z,25616.2,passed,2018-06-02T14:51:26.134Z,7358e9a93e47ac9fcaebe960c00ab8bd28e9890e,70183633,SONAR-9304 Add comments to make tests clearer,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
34,242001807,242001806,2017-06-12T12:32:57Z,2017-06-12T12:32:00.677Z,2017-06-12T12:44:03Z,25631.1,passed,2018-06-02T14:52:46.098Z,0b284e73ea86f4373e198549afbc27f12513d2e4,70199251,SONAR-9304 Add test on quality profiles for project analysis,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
35,242001808,242001806,2017-06-12T12:32:56Z,2017-06-12T12:32:00.683Z,2017-06-12T12:38:02Z,25631.2,passed,2018-06-02T14:52:46.106Z,0b284e73ea86f4373e198549afbc27f12513d2e4,70199251,SONAR-9304 Add test on quality profiles for project analysis,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
42,242002828,242002827,2017-06-12T12:35:54Z,2017-06-12T12:34:56.871Z,2017-06-12T12:47:22Z,25635.1,passed,2018-06-02T14:52:48.371Z,b492c6f375c836235b390a35b3c3caed77a89b47,70199512,(wip) replace RuleActivatorMediumTest by RuleActivatorTest,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135544,238061347,238061344,2017-05-31T19:40:36Z,2017-05-31T19:39:18.489Z,2017-05-31T19:44:46Z,25238.2,passed,2018-06-02T13:28:20.657Z,c990cd59bc753c19297e86b8404fc12798840aa5,68998200,SONAR-9344 Do not display activation actions for built-in quality profiles on rules page,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
135549,238073316,238073315,2017-05-31T20:14:35Z,2017-05-31T20:13:29.874Z,2017-05-31T20:27:10Z,25241.1,passed,2018-06-02T13:28:32.872Z,2c2c40df57223320530d4659a2cf4291cb39638f,69001951,SONAR-9304 drop columns organization_uuid and parent_kee from rules_profiles,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
135550,238073317,238073315,2017-05-31T20:14:31Z,2017-05-31T20:13:29.885Z,2017-05-31T20:19:01Z,25241.2,passed,2018-06-02T13:28:32.894Z,2c2c40df57223320530d4659a2cf4291cb39638f,69001951,SONAR-9304 drop columns organization_uuid and parent_kee from rules_profiles,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104
135577,238275939,238275938,2017-06-01T09:48:06Z,2017-06-01T09:46:59.399Z,2017-06-01T10:00:36Z,25255.1,passed,2018-06-02T13:32:57.993Z,04eb3058e2a80375fe5536c34f23052482f1302d,69062205,DO NOT MERGE - Launch only PostgreSql QA on developer branch,refs/pull/2104/merge,https://github.com/SonarSource/sonarqube/pull/2104


In [30]:
len(jobsDf[jobsDf.commitref == pull_req]) - len(jobsDf[(jobsDf.commitref == pull_req) & ~(jobsDf.commitsha.isin(pr_sha))])

0

No job retrieved with the list of PR commits is actually among the PR jobs..

### Possible problems?

These are the same commit, one on master and one on the branch. They have different shas, and this is a problem when I call compare with the api and use the sha to fidn the related job

https://github.com/SonarSource/sonarqube/commit/ed1a72601c559e86ccd96004c99157421b6bcb20

https://github.com/SonarSource/sonarqube/commit/1644704e694e384915bfc169fe330afb8d07edca